In [87]:
import tensorflow
import gym
import random
import atari_py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy



from gym import Env
from gym.spaces import Discrete, Box
import random

import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [88]:
class ShowerEnv(Env):

    def Get_state(self, time, combi):
        combi["dif_x"] = combi["Open_x"] - combi["Close_x"]
        combi["dif_y"] = combi["Open_y"] - combi["Close_y"]

        combi = combi.dropna()

        tot_labels    = []
        nsdq_train   = []
        spy_train = []

        sum_ = 0 

        for x in range (1, len(combi)):
            # if(combi.iloc[x]["Open_x"] - combi.iloc[x]["Close_x"] >=
            #     combi.iloc[x]["Open_y"] - combi.iloc[x]["Close_y"]):
            #     tot_labels.append([1])
            if(combi.iloc[x]["Open_x"] > combi.iloc[x]["Close_x"] ):
                tot_labels.append([1])
                sum_ = sum_ +1
            else:
                tot_labels.append([0]) 
                sum_ = sum_ -1
            temp_list_x =norm_fun( [combi.iloc[x-1]["Open_x"], combi.iloc[x-1]["Close_x"], 
                                    combi.iloc[x-1]["High_x"], combi.iloc[x-1]["Low_x"], 
                                    combi.iloc[x]["Open_x"]])
            temp_list_y = norm_fun([combi.iloc[x-1]["Open_y"], combi.iloc[x-1]["Close_y"], 
                                    combi.iloc[x-1]["High_y"], combi.iloc[x-1]["Low_y"], 
                                    combi.iloc[x]["Open_y"]] )

            
            nsdq_train.append( temp_list_x )
            spy_train.append( temp_list_y )

        
        nsdq_train = tf.keras.utils.normalize(nsdq_train, axis =1)
        spy_train = tf.keras.utils.normalize(spy_train, axis=1)

        tot_train = np.array(np.concatenate((nsdq_train,spy_train), axis = 1))



        tot_labels = np.array(tot_labels)
        encoder = LabelEncoder()
        encoder.fit(tot_labels)
        encoded_Y = encoder.transform(tot_labels)
        x_out = tot_train[time]
        y_out = encoded_Y[time]

        return x_out, y_out



    def __init__(self):

        self.nsdq    = pd.read_csv("NQ=F (2).csv")
        self.spy     = pd.read_csv("SPY (1).csv")
        self.time = 0

        combi   = pd.merge(self.nsdq, self.spy, on = 'Date')
        print(combi)
        print(self.spy)
        print(self.nsdq)
        
        self.action_space = Discrete(2)

        self.state = [[],[]]


        

    def norm_fun(x): 
    
        high    = x[0]
        low     = x[0]
        new_x = []

        for n in range(0, len(x)):
            if ( x[n] > high):
                high = x[n]
            if ( x[n] < low):
                low = x[n]
                
        for n in range(0, len(x)):
            new_x.append((x[n] -low) - (high - low))

        return new_x



    def step(self, action):
        self.time +=1
        self.state = self.Get_state( self.time, self.combi)

        if(state.y_out == action):
            reward = 1
        else:
            reward = -1


        if time >= 1000:
            done = True
        else:
            done = False

        return self.state, reward, done, info

    def render(self):
        pass
    def reset(self):
        self.time = 0
        return self.state

    

In [89]:
env = ShowerEnv()

Empty DataFrame
Columns: [Date, Open_x, High_x, Low_x, Close_x, Adj Close_x, Volume_x, Open_y, High_y, Low_y, Close_y, Adj Close_y, Volume_y]
Index: []
            Date        Open        High         Low       Close   Adj Close  \
0     1995-01-19   46.828125   46.875000   46.687500   46.718750   28.872732   
1     1995-01-20   46.656250   46.671875   46.453125   46.546875   28.766497   
2     1995-01-23   46.234375   46.687500   46.203125   46.687500   28.853416   
3     1995-01-24   46.671875   46.750000   46.640625   46.750000   28.892042   
4     1995-01-25   46.515625   47.046875   46.515625   46.875000   28.969303   
...          ...         ...         ...         ...         ...         ...   
6665  2021-07-12  435.429993  437.350006  434.970001  437.079987  437.079987   
6666  2021-07-13  436.239990  437.839996  435.309998  435.589996  435.589996   
6667  2021-07-14  437.399994  437.920013  434.910004  436.239990  436.239990   
6668  2021-07-15  434.809998  435.529999  432.72

In [90]:
env.action_space.sample()

0

In [91]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

AttributeError: 'ShowerEnv' object has no attribute 'combi'

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
def build_model(states, actions):
    model = Sequential()
    
    model.add(Dense(48, activation='relu', input_shape = states))
    
    model.add(Dense(21, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model


In [ ]:
model = build_model(states, actions)



In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    
    memory = SequentialMemory(limit =100, window_length =1)
    
    dqn = DQNAgent(model = model,
                   memory=memory,
                   policy=policy, 
                   
                   nb_actions = actions,
                   nb_steps_warmup =100,
                   target_model_update=1e-2
                  )
    return dqn

    
    

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-2), metrics=['mae'])
dqn.fit(env, nb_steps = 100000, visualize=False, verbose =1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))